In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
import scipy.integrate as integrate
import ipyvolume as ipv
import importlib

import os
import sys
sys.path.insert(0, os.path.abspath('pycode'))

In [ ]:
importlib.reload(ipv)

In [ ]:
%matplotlib widget

fig = plt.figure()
ax = fig.add_subplot(111)

def noise(x):
    return np.sin(x)

def fbm(octaves, lacunarity, gain, x):
    # initial values
    amplitude = 0.5;
    frequency = 1.;

    #  loop of octaves
    y = np.zeros_like(x)
    for i in range(int(octaves)):
        y += amplitude * noise(frequency*x);
        frequency *= lacunarity;
        amplitude *= gain;
    return y

def plot_fbm(octaves, lacunarity, gain):
    x = np.arange(-10, 10, 0.1)
    y = fbm(octaves, lacunarity, gain, x)
    ax.clear()
    ax.plot(x, y)

widgets.interact(plot_fbm, octaves=(1, 10, 1.0), lacunarity=(0, 10.0, 0.1), gain=(0, 2, 0.1))

In [ ]:
import os
import sys
sys.path.insert(0, os.path.abspath('pycode'))

import spherical as sph
import BRDF as BRDF

import importlib
importlib.reload(sph)
importlib.reload(BRDF)

In [ ]:
alpha = np.arange(BRDF.MIN_GGX_ALPHA, 1, 0.01)
err = np.array([BRDF.NDF_NormalizationTest(BRDF.NDF_GGX, a, num_samples=512) for a in alpha])

In [ ]:
plt.plot(alpha, np.abs(err))
plt.ylim(0, 1)

In [ ]:
import numpy as np
import os
import sys
sys.path.insert(0, os.path.abspath('pycode'))

import spherical as sph
import BRDF as BRDF

import importlib
importlib.reload(sph)
importlib.reload(BRDF)


alpha = np.arange(BRDF.MIN_GGX_ALPHA, 1, 0.2)
theta = np.arange(0, np.pi * 0.5, 0.4)
alpha, theta = np.meshgrid(alpha, theta)

err = np.zeros_like(alpha)
for i in range(alpha.shape[0]):
    for j in range(alpha.shape[1]):
        alpha_ = alpha[i][j]
        theta_ = theta[i][j]
        Wo = sph.spherical_dir(theta_, 0)
        cosThetaO = Wo[2]
        err[i][j] = BRDF.maskingFuncProjectedAreaTest(BRDF.NDF_GGX, BRDF.smithMaskingFunction, alpha_, Wo, num_samples=2048)/cosThetaO
        print(alpha_, theta_, err[i][j])


In [ ]:
import numpy as np
import os
import sys
sys.path.insert(0, os.path.abspath('pycode'))

import spherical as sph
import BRDF as BRDF

import importlib
importlib.reload(sph)
importlib.reload(BRDF)


alpha = np.arange(BRDF.MIN_GGX_ALPHA, 1, 0.01)
theta = np.arange(0, np.pi * 0.5, 0.4)
alpha, theta = np.meshgrid(alpha, theta)

err = np.zeros_like(alpha)
for i in range(alpha.shape[0]):
    for j in range(alpha.shape[1]):
        alpha_ = alpha[i][j]
        theta_ = theta[i][j]
        Wo = sph.spherical_dir(theta_, 0)
        err[i][j] = BRDF.whiteFurnaceTest(BRDF.NDF_GGX, BRDF.smithMaskingFunction, alpha_, Wo, num_samples=1024)



In [ ]:
ipv.clear()
ipv.plot_mesh(alpha, theta, np.abs(err))
ipv.show()

!export MKL_DEBUG_CPU_TYPE=5

In [ ]:
import sampling as sampling
import spherical as sph
import BRDF as BRDF
import chi2 as chi2

import importlib
importlib.reload(sampling)
importlib.reload(sph)
importlib.reload(BRDF)
importlib.reload(chi2)

samples_ = np.random.uniform(size =[2,1024*32])
ggx = sampling.NormalDistributionGGX()
alpha = 0.05
Wh = ggx.sample(alpha, samples_)
pdf = ggx.pdf(alpha, Wh)
D = ggx.eval(alpha, Wh)

def my_sample(sample):
    return ggx.sample(alpha, sample)
    return square_to_cosine_hemisphere(sample)

def my_pdf(p):
    return ggx.pdf(alpha, p)
    return square_to_cosine_hemisphere_pdf(p)

chi2Test = chi2.ChiSquareTest(
    domain=chi2.SphericalDomain(),
    sample_func=my_sample,
    pdf_func=my_pdf,
    sample_dim=2,
    sample_count=1000000,
    ires=64
)

chi2Test.tabulate_histogram()
chi2Test.tabulate_pdf(simpson_rule=True)
chi2Test.run()

fig, ax = plt.subplots(2)
fig.set_figwidth(8)
fig.set_figheight(8)
vmax = np.maximum(np.amax(chi2Test.histogram), np.amax(chi2Test.pdf))
# vmax *= 0.96
ax[0].set_title('Observed frequencies')
ax[0].imshow(chi2Test.histogram, cmap='jet', vmax=vmax)
ax[1].set_title('Expected frequencies')
ax[1].imshow(chi2Test.pdf, cmap='jet', vmax=vmax)
# print(chi2Test.histogram.shape)

# vmax = np.amax(chi2Test.histogram)
# vmax = np.maximum(vmax, np.amax(chi2Test.pdf))
# plt.imshow(chi2Test.histogram, cmap='jet', vmax=vmax)
# print(chi2Test.histogram.shape)
# # print(chi2Test.histogram.shape)
# # chi2.run()
# # chi2._dump_tables()

In [ ]:
print(vmax)
print(np.amax(chi2Test.histogram))
print(np.amax(chi2Test.pdf))

In [ ]:
from scipy.stats import chisquare


chisquare(np.maximum(chi2Test.histogram, 0.0001).flatten(), f_exp=np.maximum(chi2Test.pdf, 0.0001).flatten())

In [ ]:
x = np.arange(10)
condlist = [x<3, x>5]
print(condlist)
choicelist = [x, x**2]
np.select(condlist, choicelist)

In [ ]:
import scipy
scipy.special.gammainc?